In [1]:
import torch

# layer 비교하기 traffic 데이터를 가져오는 식으로 비교
from layers.Embed import *


In [2]:
# 관련 패키지 불러오기

import torch.nn as nn
import torch.nn.functional as F
from layers.Transformer_EncDec import Encoder, EncoderLayer
from layers.SelfAttention_Family import FullAttention, AttentionLayer
import numpy as np

In [3]:
# 변수 정리

seq_len = 96 # c_in으로 
label_len = 48
pred_len = 48
e_layers = 2
d_layers = 1
factor = 3
enc_in = 321
dec_in = 321
d_model = 512
batch_size = 32
c_out = 321
des = 'Exp'
learning_rate = 0.001
freq = 'h' # 입력빈도 - 세팅을 일일이 해야 하는 부분
dropout = 0.1
embed = 'timeF' # timeF, fixed, learned
class_strategy = 'projection'

In [4]:
enc_transformer_embedding_0 = DataEmbedding(seq_len, d_model, embed, freq, dropout) # DataEmbedding -> d_model이 반드시 짝수

In [5]:
enc_transformer_embedding_0

DataEmbedding(
  (value_embedding): TokenEmbedding(
    (tokenConv): Conv1d(96, 512, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
  )
  (position_embedding): PositionalEmbedding()
  (temporal_embedding): TimeFeatureEmbedding(
    (embed): Linear(in_features=4, out_features=512, bias=False)
  )
  (dropout): Dropout(p=0.1, inplace=False)
)

In [6]:
enc_transformer_embedding_1 = DataEmbedding_inverted(seq_len, d_model, embed, freq, dropout) # DataEmbedding -> d_model이 반드시 짝수

In [7]:
enc_transformer_embedding_1

DataEmbedding_inverted(
  (value_embedding): Linear(in_features=96, out_features=512, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [8]:
enc_transformer_embedding_2 = DataEmbedding_inverted_TCN(seq_len, d_model, embed, freq, dropout) # DataEmbedding -> d_model이 반드시 짝수

conv2 shape torch.Size([512, 512, 2])
conv2 shape torch.Size([512, 512, 2])
conv2 shape torch.Size([512, 512, 2])


In [9]:
enc_transformer_embedding_2

DataEmbedding_inverted_TCN(
  (value_embedding): TemporalConvNet(
    (network): Sequential(
      (0): TemporalBlock(
        (conv1): Conv1d(96, 512, kernel_size=(2,), stride=(1,), padding=(1,))
        (chomp1): Chomp1d()
        (relu1): ReLU()
        (dropout1): Dropout(p=0.1, inplace=False)
        (conv2): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,))
        (chomp2): Chomp1d()
        (relu2): ReLU()
        (dropout2): Dropout(p=0.1, inplace=False)
        (net): Sequential(
          (0): Conv1d(96, 512, kernel_size=(2,), stride=(1,), padding=(1,))
          (1): Chomp1d()
          (2): ReLU()
          (3): Dropout(p=0.1, inplace=False)
          (4): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,))
          (5): Chomp1d()
          (6): ReLU()
          (7): Dropout(p=0.1, inplace=False)
        )
        (downsample): Conv1d(96, 512, kernel_size=(1,), stride=(1,))
        (relu): ReLU()
      )
      (1): TemporalBlock(
        (conv1): Con

In [10]:
dummy_input = torch.randn(batch_size, enc_in, seq_len)

In [11]:
import pandas as pd

obj = 'electricity'
dataset_name = f'./dataset/{obj}/{obj}.csv'

df_base = pd.read_csv(dataset_name, encoding='utf8')


In [12]:
df_cut = df_base.drop(columns=['date'])
df_cut_list = []
for j in range(batch_size):
    df_cut_list.append(df_cut.iloc[j:seq_len+j, :])

In [13]:
df_cut_list

[       0      1      2      3      4       5     6       7      8      9  ...  \
 0   14.0   69.0  234.0  415.0  215.0  1056.0  29.0   840.0  226.0  265.0  ...   
 1   18.0   92.0  312.0  556.0  292.0  1363.0  29.0  1102.0  271.0  340.0  ...   
 2   21.0   96.0  312.0  560.0  272.0  1240.0  29.0  1025.0  270.0  300.0  ...   
 3   20.0   92.0  312.0  443.0  213.0   845.0  24.0   833.0  179.0  211.0  ...   
 4   22.0   91.0  312.0  346.0  190.0   647.0  16.0   733.0  186.0  179.0  ...   
 ..   ...    ...    ...    ...    ...     ...   ...     ...    ...    ...  ...   
 91  41.0  126.0   16.0  940.0  409.0  1547.0  28.0  1837.0  434.0  336.0  ...   
 92  27.0  136.0   20.0  906.0  349.0  1614.0  32.0  1595.0  405.0  338.0  ...   
 93  63.0  120.0  121.0  832.0  368.0  1351.0  23.0  1487.0  366.0  319.0  ...   
 94  55.0  109.0  332.0  727.0  401.0  1339.0  21.0  1403.0  322.0  311.0  ...   
 95  46.0   98.0  327.0  608.0  358.0  1189.0  18.0  1187.0  297.0  275.0  ...   
 
        311   

In [14]:
elec_input = np.array([df_cut_list[j].values for j in range(batch_size)]).astype('float')

In [15]:
elec_input = torch.tensor(elec_input, dtype=torch.float)

In [16]:
dummy_input


tensor([[[ 0.5994,  0.1401,  0.2479,  ...,  0.8924, -1.1000, -0.8960],
         [ 2.0919,  0.0868, -1.5241,  ...,  0.2208,  0.8714, -0.2966],
         [-0.3094,  0.9738, -1.4617,  ...,  0.2592, -1.8898, -0.6228],
         ...,
         [ 1.0249,  0.5741, -1.2987,  ..., -0.4407, -0.5985, -1.3073],
         [ 1.4746, -1.0919, -0.4845,  ..., -0.3942, -0.2327,  0.8215],
         [-1.0299, -1.5523,  0.1029,  ..., -1.4646, -0.6532,  0.5397]],

        [[ 1.5052,  0.2711, -0.3055,  ..., -1.6777,  1.2335, -2.0567],
         [ 0.9426,  1.5290, -0.2443,  ...,  0.9941, -0.6550, -0.1717],
         [ 0.7908, -0.5817,  0.2403,  ...,  0.3085,  0.1036,  0.1227],
         ...,
         [ 0.7400,  0.4955, -0.1071,  ..., -0.2482, -0.6013, -0.6802],
         [ 0.4244, -0.1827,  1.2290,  ..., -3.7287,  0.6457,  1.2007],
         [ 1.2893,  1.1480, -0.6570,  ...,  1.3291, -1.2343, -1.1105]],

        [[-0.0311,  0.7259,  1.7887,  ..., -0.6198,  0.4866,  0.1485],
         [-0.8453, -1.0088,  0.9046,  ..., -0

In [17]:
elec_input.size()

torch.Size([32, 96, 321])

In [18]:
output_20 = enc_transformer_embedding_2(elec_input, None)

In [19]:
output_20.size()

torch.Size([32, 321, 512])

In [20]:
output_20

tensor([[[1.3454e+01, 3.4080e+00, 1.3005e+00,  ..., 5.0008e-01,
          1.1656e+00, 1.3066e-01],
         [3.1618e+01, 1.0798e+01, 3.7621e+00,  ..., 6.4279e+00,
          0.0000e+00, 1.8881e+01],
         [4.8735e+01, 3.5219e+00, 5.5729e+00,  ..., 7.1800e+00,
          3.1803e+00, 1.5872e+00],
         ...,
         [1.0843e+04, 0.0000e+00, 1.2907e+03,  ..., 2.7821e+03,
          4.2446e+03, 0.0000e+00],
         [4.9234e+03, 1.4485e+02, 0.0000e+00,  ..., 1.0469e+03,
          2.3856e+02, 2.8450e+02],
         [0.0000e+00, 1.0738e+02, 0.0000e+00,  ..., 3.1285e+02,
          1.4933e+03, 0.0000e+00]],

        [[1.5755e+01, 1.5029e+00, 0.0000e+00,  ..., 2.5906e-01,
          0.0000e+00, 0.0000e+00],
         [4.1148e+01, 1.7052e+00, 0.0000e+00,  ..., 4.3639e+00,
          1.4482e+00, 0.0000e+00],
         [4.5996e+01, 0.0000e+00, 6.0991e+00,  ..., 0.0000e+00,
          3.4842e+01, 2.7037e+01],
         ...,
         [8.4145e+03, 1.1106e+02, 0.0000e+00,  ..., 2.6757e+03,
          3.891

In [21]:
output_10 = enc_transformer_embedding_1(elec_input, None)

In [22]:
output_10.size()

torch.Size([32, 321, 512])

In [23]:
output_10

tensor([[[ 1.8756e+01,  2.2122e+01,  2.2250e+01,  ..., -3.6869e+00,
           2.0533e+01, -2.1557e+01],
         [ 7.7018e-01,  6.0249e+01,  6.8835e+01,  ...,  4.9185e+01,
           6.1261e+01, -4.7543e+01],
         [-2.0889e+01,  7.8173e+01,  5.0099e+01,  ...,  4.6817e+01,
           6.1698e+01, -1.0423e+01],
         ...,
         [-5.7650e+02,  1.0391e+03,  1.5442e+03,  ...,  1.2894e+03,
           8.4253e+02, -1.2677e+03],
         [ 9.4327e+01,  4.1521e+02,  0.0000e+00,  ...,  1.7790e+02,
           2.3451e+02, -2.1843e+02],
         [-2.2016e+02,  1.6786e+03,  2.1679e+03,  ...,  1.4021e+03,
           1.2650e+03, -1.3896e+03]],

        [[ 5.3157e+00,  3.1141e+01,  9.2989e+00,  ..., -1.2902e+00,
           2.6620e+01, -2.3921e+01],
         [-1.5787e+00,  5.8454e+01,  9.3739e+01,  ...,  5.9210e+01,
           4.3091e+01, -3.4143e+01],
         [-5.3031e+01,  7.9044e+01,  1.9571e+01,  ...,  3.1404e+01,
           2.8940e+00,  3.1377e+01],
         ...,
         [-6.7130e+02,  6

In [24]:
num_small_elements_20 = torch.sum(abs(output_20) < 0.0001).item()
num_small_elements_10 = torch.sum(abs(output_10) < 0.0001).item()

num_small_elements_20, num_small_elements_10, 32*321*512

(942810, 525958, 5259264)

In [25]:
# temporal block size
temporal_conv_net_ex = TemporalConvNet(seq_len, [d_model]*3)

conv2 shape torch.Size([512, 512, 2])
conv2 shape torch.Size([512, 512, 2])
conv2 shape torch.Size([512, 512, 2])


In [26]:
output_ex = temporal_conv_net_ex(elec_input)

In [27]:
output_ex.size()

torch.Size([32, 512, 321])

In [ ]:
output_temporal = temporal_block(dummy_input)

In [ ]:
output_temporal

In [ ]:
output_temporal.size()

In [ ]:
temporal_conv_net = TemporalConvNet1(96, [512, 512, 512])

In [ ]:
output_temporal_conv_net = temporal_conv_net(dummy_input_0)

In [ ]:
output_temporal_conv_net

In [ ]:
output_temporal_conv_net.size()

In [ ]:
tcn_res = TCN(96, 48, [512]*3, 2, 0.2)

In [ ]:
tcn_res

In [ ]:
tcn_res(dummy_input_0)

In [ ]:
tcn_res(dummy_input_0).size()

In [ ]:
tmp_conv_net = TemporalConvNet(seq_len, [d_model]*3, 2, 0.005)

In [ ]:
tmp_conv_net(dummy_input_0).size()

In [ ]:
enc_transformer_embedding_2 = DataEmbedding_inverted_TCN(seq_len, d_model, embed, freq, dropout) # DataEmbedding -> d_model이 반드시 짝수

In [ ]:
enc_transformer_embedding_2

In [ ]:
enc_transformer_embedding_2(dummy_input_0, None)

In [ ]:
TemporalConvNet(c_in, [d_model]*3) 

In [ ]:
import numpy as np

In [ ]:
np.linspace(4,12,4)

### TCN 레이어 차원 검색

In [4]:
tcn_sample = TCN(d_model, pred_len, [d_model]*3, 2, dropout)

In [5]:
temp = torch.randn(batch_size, enc_in, d_model)

In [6]:
temp.shape

torch.Size([32, 321, 512])

In [7]:
tcn_sample(temp).shape

torch.Size([32, 321, 48])

In [8]:
lin_sample = nn.Linear(d_model, pred_len, bias=True)

In [9]:
lin_sample(temp).shape

torch.Size([32, 321, 48])